## Legacy reporter 


A simple reporter that only saves conformations in a "legacy" format - using polymerutils.save 


In [1]:
import polychrom
import numpy as np 
import warnings
import h5py 
import glob
from polychrom.simulation import Simulation
import polychrom.starting_conformations
import polychrom.forces, polychrom.forcekits
import simtk.openmm 
import os 
import shutil
import polychrom.polymerutils

### Loading reporter and utils from a hdf5_format module 


In [2]:
from polychrom.legacy.legacy_format import LegacyReporter

### Making a simulation and passing a reporter 

In [3]:
%rm  test_legacy/*
data = polychrom.starting_conformations.grow_cubic(10000,30)

"""
Here we created a hdf5Reporter attached to a foler test, and we are saving 5 blocks per file 
(you should probalby use 50 here or 100. 5 is just for a showcase)
"""
reporter = LegacyReporter(folder="test_legacy")


"""
Passing a reporter to the simulation object - many reporters are possible, and more will be added in a future
"""
sim = Simulation(N=10000, error_tol=0.001, collision_rate=0.01, integrator ="variableLangevin", platform="CPU", 
                reporters=[reporter])
sim.set_data(data)
sim.add_force(polychrom.forcekits.polymer_chains(sim))
sim._apply_forces()
sim.add_force(polychrom.forces.spherical_confinement(sim, density=0.1))


for i in range(19):        
    """
    Here we pass two extra records: a string and an array-like object.
    First becomes an attr, and second becomes an HDF5 dataset
    """
    sim.do_block(10, save_extras={"eggs": "I don't eat green eggs and ham!!!", "spam":[1,2,3]})

"""
Here we are not forgetting to dump the last set of blocks that the reporter has. 
We have to do it at the end of every simulation. 

I tried adding it to the destructor to make it automatic,
but some weird interactions with garbage collection made it not very useable. 
"""
reporter.dump_data()




INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:adding force polynomial_repulsive 2


Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 9999


INFO:root:Particles loaded. Potential energy is 0.050724
INFO:root:block    0 pos[1]=[13.9 14.0 13.9] dr=0.21 t=0.9ps kin=8.17 pot=3.97 Rg=11.017 dt=24.2fs dx=15.43pm 
INFO:root:block    1 pos[1]=[13.9 14.0 14.1] dr=0.17 t=1.2ps kin=3.86 pot=8.82 Rg=11.018 dt=20.7fs dx=9.07pm 
INFO:root:block    2 pos[1]=[13.8 14.0 14.1] dr=0.09 t=1.4ps kin=7.72 pot=5.01 Rg=11.020 dt=24.7fs dx=15.36pm 
INFO:root:block    3 pos[1]=[13.8 13.9 14.0] dr=0.15 t=1.6ps kin=6.35 pot=6.38 Rg=11.025 dt=22.4fs dx=12.61pm 
INFO:root:block    4 pos[1]=[13.7 13.9 14.0] dr=0.11 t=1.8ps kin=6.67 pot=6.03 Rg=11.030 dt=22.1fs dx=12.77pm 
INFO:root:block    5 pos[1]=[13.7 13.9 14.0] dr=0.12 t=2.0ps kin=6.77 pot=5.91 Rg=11.036 dt=22.1fs dx=12.87pm 
INFO:root:block    6 pos[1]=[13.7 13.9 14.1] dr=0.12 t=2.3ps kin=6.97 pot=5.69 Rg=11.041 dt=22.1fs dx=13.05pm 
INFO:root:block    7 pos[1]=[13.7 13.9 14.1] dr=0.13 t=2.5ps kin=7.22 pot=5.41 Rg=11.047 dt=22.1fs dx=13.29pm 
INFO:root:block    8 pos[1]=[13.7 13.8 14.2] dr=0.12 t=2

### This is a list of files created in the trajectory folder

In [4]:
!ls -la test_legacy

total 3808
drwxrwxr-x 2 magus magus   4096 Sep  2 19:38 .
drwxrwxr-x 5 magus magus   4096 Sep  2 19:37 ..
-rw-rw-r-- 1 magus magus 201982 Sep  2 19:38 block0.dat
-rw-rw-r-- 1 magus magus 201939 Sep  2 19:38 block10.dat
-rw-rw-r-- 1 magus magus 201930 Sep  2 19:38 block11.dat
-rw-rw-r-- 1 magus magus 201931 Sep  2 19:38 block12.dat
-rw-rw-r-- 1 magus magus 201922 Sep  2 19:38 block13.dat
-rw-rw-r-- 1 magus magus 201906 Sep  2 19:38 block14.dat
-rw-rw-r-- 1 magus magus 201894 Sep  2 19:38 block15.dat
-rw-rw-r-- 1 magus magus 201900 Sep  2 19:38 block16.dat
-rw-rw-r-- 1 magus magus 201898 Sep  2 19:38 block17.dat
-rw-rw-r-- 1 magus magus 201897 Sep  2 19:38 block18.dat
-rw-rw-r-- 1 magus magus 201955 Sep  2 19:38 block1.dat
-rw-rw-r-- 1 magus magus 201964 Sep  2 19:38 block2.dat
-rw-rw-r-- 1 magus magus 201968 Sep  2 19:38 block3.dat
-rw-rw-r-- 1 magus magus 201968 Sep  2 19:38 block4.dat
-rw-rw-r-- 1 magus magus 201951 Sep  2 19:38 block5.dat
-rw-rw-r-- 1 magus magus 201955 Sep  2 19:38 

In [5]:
polychrom.polymerutils.load("test_legacy/block5.dat")

array([[13.737, 13.871, 14.036],
       [14.139, 14.372, 14.791],
       [14.281, 13.777, 15.768],
       ...,
       [12.975, 13.068, 13.005],
       [12.838, 13.504, 13.85 ],
       [13.389, 14.242, 14.17 ]])